# HTML to dataframe
This notebook converts the City of Detroit Directory of Depts. contact website into a pandas data frame then save it in a csv file

In [2]:
# Import
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import FirefoxOptions
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By

In [3]:
def get_driver(choice):
    '''
    Initialize a Selenium WebDriver based on choice.

    Parameters:
    choice (str): The name of the browser to use. Options are 'Chrome' or 'Firefox'.

    Returns:
    WebDriver: An instance of the specified WebDriver (Chrome or Firefox). 
                If the driver cannot be started in normal mode, it falls back to headless mode.
                Returns None if the choice is not recognized.

    Notes:
    - The function attempts to start the driver in regular mode first. 
    - If there is an error starting the driver, it will attempt to start it in headless mode.
    '''
    if choice == 'Chrome':
        opts = webdriver.ChromeOptions()
        try: # Try starting a regular driver
            driver = webdriver.Chrome(options=opts)
        except: # If not possible, start a headless/invisible driver
            opts.add_argument("--headless")
            driver = webdriver.Chrome(options=opts)
            print('Headless/Invisible Driver')
        return driver
    if choice == 'Firefox': 
        opts = webdriver.FirefoxOptions()
        try: # Try starting a regular driver
            driver = webdriver.Firefox(options=opts)
        except: # If not possible, start a headless/invisible driver
            opts.add_argument("--headless")
            driver = webdriver.Firefox(options=opts)
            print('Headless/Invisible Driver')
        return driver
    return None

In [4]:
# Boot up a Firefox webdriver
driver = get_driver('Firefox') # This might take a while to boot up, if this doesn't work, try Chrome
time.sleep(3)

url = "https://publish.smartsheet.com/9def816c9e6a4a4395d2903039bf714d" # go to the website 
driver.get(url)
time.sleep(2)

In [7]:
element = driver.find_element(By.TAG_NAME, 'table') # Get the main table of the website

df = pd.read_html('https://publish.smartsheet.com/9def816c9e6a4a4395d2903039bf714d') # convert the html table into pd data frame
df[0].drop(['Web Address', 'Department'], axis=1, inplace=True) # drop the two columns
df[0] = df[0].map(lambda x: np.nan if isinstance(x, str) and len(x) == 1 else x) # convert all one-character strings into np.nan
# df[0] = df[0].loc[:, ~df[0].columns.str.contains('^Unnamed')] # drop the 'Unnamed: 0' column
df[0]['Name'] = df[0]['Name'].replace(['General Information','Administration'], np.nan) # convert unnecessary info in the name column to na
df[0][['Name', 'Title']] = df[0]['Name'].str.split('; ', n=1, expand=True) # split name and title to different columns
df[0] = df[0].fillna(value=np.nan) # fill all null value with np.nan
official = df[0].dropna(how='all')[:-1] # drop row with all np.nan values and the last row which

In [8]:
official

,Unnamed: 0,Name,Location / Suite,Phone Number,Fax,Email,Title
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,"APPEALS and HEARINGS, DEPT.",NaN,NaN,NaN,NaN,NaN
4,3.0,Customer Service,"CAYMC, Ste. 106 Detroit, 48226",(313)224-0098,NaN,dah_cs@detroitmi.gov,NaN
...,...,...,...,...,...,...,...
1492,768.0,NaN,NaN,NaN,NaN,NaN,NaN
1493,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1494,769.0,NaN,NaN,NaN,NaN,NaN,NaN
1495,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
official.to_csv('contact.csv') # convert df into csv file